<a href="https://colab.research.google.com/github/kridtapon/RVI-Coppock-Synergy/blob/main/RVI_Coppock_Synergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninsta

In [33]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Relative Vigor Index (RVI)
def calculate_rvi(data, period=10):
    """
    Calculate Relative Vigor Index (RVI).
    """
    close_open_diff = data['Close'] - data['Open']
    high_low_diff = data['High'] - data['Low']

    numerator = close_open_diff.rolling(window=period).mean()
    denominator = high_low_diff.rolling(window=period).mean()

    rvi = numerator / denominator
    return rvi

# Function to calculate Coppock Curve
def calculate_coppock(data, wma_period=10, roc1_period=14, roc2_period=11):
    """
    Calculate Coppock Curve.
    """
    roc1 = data['Close'].pct_change(roc1_period) * 100
    roc2 = data['Close'].pct_change(roc2_period) * 100

    coppock = (roc1 + roc2).rolling(window=wma_period).mean()
    return coppock

# Define the stock symbol and time period
symbol = 'APO'  # APO
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close','Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate RVI and Coppock Curve
df['RVI'] = calculate_rvi(df, period=9)
df['Coppock'] = calculate_coppock(df, wma_period=5, roc1_period=7, roc2_period=19)

# Define Entry and Exit signals based on RVI and Coppock Curve
df['Entry'] = (df['Coppock'] > 0) & (df['RVI'] > 0)  # Both Coppock and RVI are positive
df['Exit'] = (df['Coppock'] < 0) & (df['RVI'] < 0)   # Both Coppock and RVI are negative

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()



[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           388458.276449
Total Return [%]                       288.458276
Benchmark Return [%]                   246.537986
Max Gross Exposure [%]                      100.0
Total Fees Paid                      10876.064844
Max Drawdown [%]                        27.424143
Max Drawdown Duration                       272.0
Total Trades                                   29
Total Closed Trades                            29
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            51.724138
Best Trade [%]                          46.245484
Worst Trade [%]                         -9.588544
Avg Winning Trade [%]                   15.935837
Avg Losing Trade [%]                    -5.149099


In [32]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from itertools import product

# Function to calculate Relative Vigor Index (RVI)
def calculate_rvi(data, period=10):
    close_open_diff = data['Close'] - data['Open']
    high_low_diff = data['High'] - data['Low']

    numerator = close_open_diff.rolling(window=period).mean()
    denominator = high_low_diff.rolling(window=period).mean()

    rvi = numerator / denominator
    return rvi

# Function to calculate Coppock Curve
def calculate_coppock(data, wma_period=10, roc1_period=14, roc2_period=11):
    roc1 = data['Close'].pct_change(roc1_period) * 100
    roc2 = data['Close'].pct_change(roc2_period) * 100

    coppock = (roc1 + roc2).rolling(window=wma_period).mean()
    return coppock

# Define the stock symbol and time period
symbol = 'APO'  # APO
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close','Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Define a function to backtest based on specific parameters
def backtest_rvi_coppock(period_rvi, wma_period, roc1_period, roc2_period):
    # Calculate RVI and Coppock Curve with the current parameters
    df['RVI'] = calculate_rvi(df, period=period_rvi)
    df['Coppock'] = calculate_coppock(df, wma_period=wma_period, roc1_period=roc1_period, roc2_period=roc2_period)

    # Define Entry and Exit signals based on RVI and Coppock Curve
    df['Entry'] = (df['Coppock'] > 0) & (df['RVI'] > 0)  # Both Coppock and RVI are positive
    df['Exit'] = (df['Coppock'] < 0) & (df['RVI'] < 0)   # Both Coppock and RVI are negative

    # Filter data for the test period (2020-2025)
    df_filtered = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df_filtered['Close'],
        entries=df_filtered['Entry'],
        exits=df_filtered['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    return portfolio.stats()

# Define ranges for each parameter
rvi_period_range = range(5, 30, 2)
wma_period_range = range(5, 30, 2)
roc1_period_range = range(5, 30, 2)
roc2_period_range = range(5, 30, 2)

# Generate all combinations of parameter values
param_combinations = product(rvi_period_range, wma_period_range, roc1_period_range, roc2_period_range)

# Store results for comparison
results = []

# Perform grid search
for params in param_combinations:
    period_rvi, wma_period, roc1_period, roc2_period = params
    stats = backtest_rvi_coppock(period_rvi, wma_period, roc1_period, roc2_period)
    results.append((params, stats['Total Return [%]']))

# Sort the results based on the total return (or any other metric you'd like to optimize)
best_params = max(results, key=lambda x: x[1])
print(f"Best Parameters: {best_params[0]}")
print(f"Best Total Return: {best_params[1]}")


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Best Parameters: (9, 5, 7, 19)
Best Total Return: 288.45827644921377
